# Day 8

## GENERIC SETUP

In [1]:
# General imports
import pytest
import ipytest
import time
import functools

# Setup ipytest
ipytest.autoconfig()

# Setup nb_black
%load_ext nb_black

# Decorator to time solutions
def timer(func):
    """
    Wrapper function.
    Print the runtime of the decorated function.
    """

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()  # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()  # 2
        run_time = end_time - start_time  # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value

    return wrapper_timer

<IPython.core.display.Javascript object>

## SOLUTION SETUP

In [5]:
# Solution-specific imports

# What day do we solve? Used to identify the input datafile, integer value
DAY = 8

<IPython.core.display.Javascript object>

#### I/O functions

In [25]:
def get_input():
    with open(f"../data/{DAY}.txt", "r") as f:
        return split_input(f.read())


def split_input(input_raw):
    """Strip trailing newline, then split on newline"""
    return [line.strip() for line in input_raw.rstrip().split("\n")]

<IPython.core.display.Javascript object>

#### Pytest input data

In [26]:
# Sample input
@pytest.fixture
def dummy_input():
    return """\
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
"""

<IPython.core.display.Javascript object>

## Solution A

In [53]:
@timer
def solve_A(lines):
    """
    Continue executing the operations until we encounter a before-seen instruction.
    """
    # Initialisation
    acc_val = 0  # accumulator initial value
    instr_ix = 0  # initial instruction index

    # Generate list of seen instructions as bool
    num_instructions = len(lines)
    instr_seen = [False for x in range(num_instructions)]

    # Keep going, unless we go out of bounds
    while True and (instr_ix < num_instructions):
        # Break if we've seen this instruction before
        if instr_seen[instr_ix]:
            break

        # Set current line to True in seen instructions
        instr_seen[instr_ix] = True

        # Perform the operations
        operation = lines[instr_ix].split(" ")

        if operation[0] == "jmp":
            instr_ix += int(operation[1])
        else:
            instr_ix += 1
            if operation[0] == "acc":
                acc_val += int(operation[1])
            elif operation[0] == "nop":
                pass  # nop does nothing yet

        # END OF WHILE LOOP

    # While loop concluded, return acc_val
    return acc_val

<IPython.core.display.Javascript object>

#### Tests

In [54]:
%%run_pytest[clean] -qq

def test_A(dummy_input):
    assert solve_A(split_input(dummy_input)) == 5

<IPython.core.display.Javascript object>

.                                                                                                                                                                                                                                      [100%]


<IPython.core.display.Javascript object>

#### OUTPUT

In [55]:
solve_A(get_input())

Finished 'solve_A' in 0.0002 secs


2014

<IPython.core.display.Javascript object>

## Solution B

In [70]:
def run_code(lines):
    """
    Continue executing the operations until we encounter a before-seen instruction.
    Minor extension of solve_A: adds loop_flag to output.
    
    Returns: tuple of (accumulator value, boolean indicator of bootloop)
    """
    # Initialisation
    acc_val = 0  # accumulator initial value
    instr_ix = 0  # initial instruction index
    loop_flag = False

    # Generate list of seen instructions as bool
    num_instructions = len(lines)
    instr_seen = [False for x in range(num_instructions)]

    # Keep going, unless we go out of bounds
    while True and (instr_ix < num_instructions):
        # Break if we've seen this instruction before
        if instr_seen[instr_ix]:
            loop_flag = True
            break

        # Set current line to True in seen instructions
        instr_seen[instr_ix] = True

        # Perform the operations
        operation = lines[instr_ix].split(" ")

        if operation[0] == "jmp":
            instr_ix += int(operation[1])
        else:
            instr_ix += 1
            if operation[0] == "acc":
                acc_val += int(operation[1])
            elif operation[0] == "nop":
                pass  # nop does nothing yet

        # END OF WHILE LOOP

    # While loop has concluded: return acc_val and loop_flag
    return acc_val, loop_flag

@timer
def solve_B(lines):
    """
    Loop over the lines, replacing nop with jmp and vice versa.
    If we find a solution that doesn't loop, break from for-loop
    and return the acc_val
    """
    # Keep a copy of original input, initialise return val
    original_lines = lines.copy()
    acc_val = "INIT"
    
    for ix, line in enumerate(lines):
        operation = line.split(" ")
        
        # Replace nop with jmp if value != 0
        if operation[0] == "nop" and operation[1] not in ["+0", "-0"]:
            lines[ix] = line.replace("nop", "jmp")
            
        # OR: replace jmp with nop
        elif operation[0] == "jmp":
            lines[ix] = line.replace("jmp", "nop")
            
        # OR: don't change the line, go to next iteration of for-loop
        else:
            continue
            
        # Test the program for correct execution
        acc_val_experiment, loop_flag = run_code(lines)
        
        # If not bootloop: We're done! break out of for-loop and return acc_val
        if not loop_flag:
            acc_val = acc_val_experiment
            break    
        # If bootloop: restore code, continue to next line experiment
        else:
            lines = original_lines.copy()
            acc_val
            
    return acc_val


<IPython.core.display.Javascript object>

#### Tests

In [71]:
%%run_pytest[clean] -qq

def test_B(dummy_input):
    assert solve_B(split_input(dummy_input)) == 8

<IPython.core.display.Javascript object>

.                                                                                                                                                                                                                                      [100%]


<IPython.core.display.Javascript object>

#### OUTPUT

In [72]:
solve_B(get_input())

Finished 'solve_B' in 0.0222 secs


2251

<IPython.core.display.Javascript object>